In [33]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
import tldextract
import numpy as np
import matplotlib.pylab as plt
import os
import re

In [34]:
def on_sale_chk(text):
    if len(text)<1:
        return False
    return 'domain' in text and 'sale' in text

def on_parked_chk(text):
    if len(text)<1:
        return True
    return 'domain' in text and 'park' in text

def on_Parked(text):
    if len(text)<1:
        return True
    return (('website' in text or 'content' in text) and 'unavailable' in text) or ('will' in text and 'soon' in text)

In [35]:
#returns html contents, textual character length, website size, status code, parked or on sale

def soupFromUrl(scrapeUrl):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; zh-CN) AppleWebKit/533+ (KHTML, like Gecko)'}
    try:
        req = requests.get(scrapeUrl, headers=headers, timeout=5)
        # print(req.status_code)
        req.close()
        if req.status_code == 200:
            # print(bs(req.text, 'html.parser').get_text().strip().replace('\n',' '))
            soup = bs(req.text,'lxml')

            # print(soup)

            text = ''

            if soup.body:
                text = re.sub(r'[^\w]', ' ',soup.body.get_text(' ', strip=True).lower())

            # print(soup)

            # print('text',text)
            # print(bs(req.text, 'html.parser'))
            # return [bs(req.text, 'html.parser'),len(req.text), len(req.content), req.status_code]
            # print([len(req.content), len(text), req.status_code, 0+(on_sale_chk(text) or on_Parked(text) or on_parked_chk(text))])
            return [len(req.content), len(text), req.status_code, 0+(on_sale_chk(text) or on_Parked(text) or on_parked_chk(text))]
        else:
            # return [-1,0,0,req.status_code]
            return [0,0,req.status_code,0]
    except:
        # return [-1,0,0,-1]
        return [0,0,-1,0]

In [36]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; zh-CN) AppleWebKit/533+ (KHTML, like Gecko)'}
req = requests.get('https://www.lycos.com/', headers=headers, timeout=5)
print(req.status_code)
req.close()
if req.status_code == 200:
    # print(bs(req.text, 'html.parser').get_text().strip().replace('\n',' '))
    soup = bs(req.text,'lxml')

    # print(soup)
    text = ''

    if soup.body:
        text = re.sub(r'[^\w]', ' ',soup.body.get_text(' ', strip=True).lower())

    print(soup)
    print('text',text)
    print([len(req.content), len(text), req.status_code, 0+(on_sale_chk(text) or on_Parked(text) or on_parked_chk(text))])

200
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
<meta content="Lycos, Inc., is a web search engine and web portal established in 1994, spun out of Carnegie Mellon University. Lycos also encompasses a network of email, webhosting, social networking, and entertainment websites." name="description"/>
<meta content="" name="author"/>
<link href="https://ly.lygo.net/static/lycos/img/favicon.ico" rel="icon" type="image/png"/>
<title>Lycos.com</title>
<link href="//fonts.googleapis.com/css?family=Lato:400,300,300italic,400italic,700,700italic" rel="stylesheet" type="text/css"/>
<link href="/css/in/fonts.css" rel="stylesheet" type="text/css"/>
<link href="https://ly.lygo.net/static/lycos/css/in/font-awesome.css" rel="stylesheet" type="text

In [37]:
# print(soupFromUrl('https://www.delinian.com/'))
# print(soupFromUrl('https://www.makecashonline.com/'))
print(soupFromUrl('https://www.lycos.com/'))

[13611, 328, 200, 0]


In [38]:
UCI_url = list(pd.read_csv('../Dataset/URL Data/UCI Dataset.csv',delimiter='\t')['URL'])
UCI_url[:10]

['www.fullonsms.com',
 'www.fullonsms.com',
 'www.dbuk.net',
 'www.SMS.ac/u/bootydelious',
 'www.areyouunique.co.uk',
 'www.4-tc.biz',
 'www.Ldew.com1win150ppmx3age16',
 'info:www.100percent-real.com',
 'www.Ldew.com',
 'www.regalportfolio.co.uk']

In [39]:
import re

def find_first_slash_preceded_by_number(s):
    # Regular expression to find the first instance of a number followed by '/'
    match = re.search(r'\d+/', s)
    
    if match:
        return match.start() + len(match.group()) - 1  # Return the index of '/'
    else:
        return -1  # Return -1 if no match is found

# Example usage
string = "example77/test 88/test2 99/test3"
index = find_first_slash_preceded_by_number(string)
print(index)  # Outputs the index of the first '/' preceded by a number

9


In [40]:
unique_UCI_url = set(UCI_url )

In [41]:
for idx,i in enumerate(unique_UCI_url):
    if '..' in i:
        if 'www' in i:
            unique_UCI_url[idx] = ''
        else:
            unique_UCI_url[idx] = unique_UCI_url[idx].replace('..','.')

In [42]:
unique_UCI_url = [i for i in unique_UCI_url if i!='']

In [43]:
UCI_dataset = {'ham':list(pd.read_csv('../Dataset/URL Data/UCI Dataset Ham.csv',delimiter='\t')['URL']),'spam':list(pd.read_csv('../Dataset/URL Data/UCI Dataset Spam.csv',delimiter='\t')['URL'])}

In [44]:
UCI_url_in_ham = [0]*len(unique_UCI_url)
UCI_url_in_spam = [0]*len(unique_UCI_url)

for idx,i in enumerate(unique_UCI_url):
    for j in UCI_dataset['ham']:
        if not isinstance(j, str):
            continue
        if i in j:
            UCI_url_in_ham[idx] = 1
            break
    for j in UCI_dataset['spam']:
        if not isinstance(j, str):
            continue
        if i in j:
            UCI_url_in_spam[idx] = 1
            break

print(UCI_url_in_ham.count(1))
print(UCI_url_in_spam.count(1))

1
54


In [45]:
common_urls = []
for i in range(len(UCI_url_in_ham)):
    if UCI_url_in_ham[i]==UCI_url_in_spam[i]:
        common_urls.append(unique_UCI_url[i])

len(common_urls)

4

In [46]:
UCI_dataset['Unique Url'] = unique_UCI_url

In [47]:
import tldextract

def FQDN(Url):
    
    url_extract_res = tldextract.extract(Url)
    fqdn = ''
    if url_extract_res.subdomain:
        fqdn = url_extract_res.subdomain + '.' + url_extract_res.domain + '.' + url_extract_res.suffix
        # fqdn = url_extract_res.domain + '.' + url_extract_res.suffix
    else:
        fqdn = url_extract_res.domain + '.' + url_extract_res.suffix
    
    return fqdn

In [48]:
UCI_dataset['FQDN'] = [FQDN(i) for i in UCI_dataset['Unique Url']]

len(set(UCI_dataset['FQDN']))

54

In [49]:
a = soupFromUrl('https://facebook.com')

print(a)

[74351, 645, 200, 0]


The below query last ran on 7 Novemebr 2024

In [50]:
website_size, text_content_length, status_code, parked = [],[],[],[]

for i in UCI_dataset['FQDN']:
    a = soupFromUrl('https://'+i)

    website_size.append(a[0])
    text_content_length.append(a[1])
    status_code.append(a[2])
    parked.append(a[3])


# parked = [0]*len(UCI_dataset)

# for idx,i in enumerate(UCI_dataset['FQDN']):
#     if UCI_dataset['Status Code'][idx]==200:
#         a = soupFromUrl('https://'+i)
#         parked[idx] = a[3]
#         # break

# print(parked)

In [51]:
UCI_dataset['Website Size in KB'] = website_size
UCI_dataset['Website Textual Content Length'] = text_content_length
UCI_dataset['Status Code'] = status_code

UCI_dataset['Parked'] = parked

In [52]:
for i in UCI_dataset:
    print(len(UCI_dataset[i]))

2
106
59
59
59
59
59
59


In [53]:
UCI_dataset['ham'] = UCI_url_in_ham
UCI_dataset['spam'] = UCI_url_in_spam

In [54]:
UCI_dataset

{'ham': [0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'spam': [1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1],
 'Unique Url': ['www.txt43.com',
  'www.fullonsms.com',
  'www.areyouunique.co.uk',
  'www.Ldew.com1win150ppmx3age16',
  'www.SMS.ac/u/bootydelious',
  'www.ldew.com1win150ppmx3age16',
  'info:www.100percent-real.com',
  'www.clubmoby.com',
  'www.SMS.ac/u/natalie2k9',
  'www.txt82228.com.',
  'www.ringtoneking.co.uk',
  'www.clubzed.co.uk',
  'www.telediscount.co.uk',
  'http://www.bubblet

In [55]:
# UCI_dataset = pd.read_csv('../Dataset/URL Data/UCI Websites Analysis.csv')
UCI_dataset = pd.DataFrame.from_dict(UCI_dataset)
UCI_dataset.head()

,ham,spam,Unique Url,FQDN,Website Size in KB,Website Textual Content Length,Status Code,Parked
0,0,1,www.txt43.com,www.txt43.com,114,0,200,1
1,1,0,www.fullonsms.com,www.fullonsms.com,1178,0,200,1
2,0,1,www.areyouunique.co.uk,www.areyouunique.co.uk,0,0,-1,0
3,0,1,www.Ldew.com1win150ppmx3age16,www.Ldew.com1win150ppmx3age16.,0,0,-1,0
4,0,1,www.SMS.ac/u/bootydelious,www.SMS.ac,1034,0,200,1


In [56]:
UCI_dataset['Status Code'].value_counts()

Status Code
-1      45
 200    14
Name: count, dtype: int64

In [57]:
numbers_to_replace = [501,403, 401]

# Value to replace with
new_value = 200

# Update the column
UCI_dataset.loc[UCI_dataset['Status Code'].isin(numbers_to_replace), 'Status Code'] = new_value

In [58]:
UCI_dataset['Status Code'].value_counts()

Status Code
-1      45
 200    14
Name: count, dtype: int64

In [59]:
print(len(UCI_dataset[(UCI_dataset['Status Code']==200) & (UCI_dataset['ham']==1)]))
print(len(UCI_dataset[(UCI_dataset['Status Code']==200) & (UCI_dataset['spam']==1)]))

1
12


In [60]:
UCI_dataset['Parked'].value_counts()

Parked
0    49
1    10
Name: count, dtype: int64

In [61]:
print(len(UCI_dataset[(UCI_dataset['Parked']==1) & (UCI_dataset['ham']==1)]))
print(len(UCI_dataset[(UCI_dataset['Parked']==1) & (UCI_dataset['ham']==0)]))

1
9


In [62]:
UCI_dataset.head()

,ham,spam,Unique Url,FQDN,Website Size in KB,Website Textual Content Length,Status Code,Parked
0,0,1,www.txt43.com,www.txt43.com,114,0,200,1
1,1,0,www.fullonsms.com,www.fullonsms.com,1178,0,200,1
2,0,1,www.areyouunique.co.uk,www.areyouunique.co.uk,0,0,-1,0
3,0,1,www.Ldew.com1win150ppmx3age16,www.Ldew.com1win150ppmx3age16.,0,0,-1,0
4,0,1,www.SMS.ac/u/bootydelious,www.SMS.ac,1034,0,200,1


In [63]:
UCI_dataset.to_csv('../Dataset/URL Data/UCI Websites Analysis.csv', index=None)